# Tensorflow 함수와 그래프
간단한 예시로 세 제곱을 계산하는 함수를 만들어서 **텐서플로 함수**로 변환해 보겠음

In [1]:
import tensorflow as tf

In [2]:
def cube(x):
    return x ** 3

In [3]:
cube(2)

8

In [4]:
cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [5]:
tf_cube = tf.function(cube)
tf_cube

**tf.function** 을 사용하여 파이썬 함수를 **텐서플로 함수로 변환할 수 있음.**  
이 텐서플로 함수는 원래 파이썬 함수처럼 사용할 수 있고 동일한(타입은 텐서인) 결과를 반환함.

In [6]:
tf_cube(2)

<tf.Tensor: shape=(), dtype=int32, numpy=8>

In [7]:
tf_cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

>실제로는 이것보다 더 간단하게 **tf.function 데커레이터** 가 널리 사용됨

In [8]:
@tf.function
def tf_cube(x):
    return x**3

In [9]:
tf_cube(2)

<tf.Tensor: shape=(), dtype=int32, numpy=8>

> 원본 파이썬 함수가 필요할 때는 텐서플로함수의 **python_function** 속성으로 참조할 수 있음.

In [10]:
tf_cube.python_function(2)

8

> 텐서플로는 그래프 내의 연산을 효율적으로 실행함.  
일반적으로 텐서플로 함수는 원본 파이썬 함수보다 훨씬 빠르게 실행됨.  
즉 **파이썬 함수를 빠르게 실행하려면 텐서플로 함수로 변환하는 것을 강력추천**

또한 사용자 정의 손실, 지표, 층 등을 케라스 모델에 사용할 때 케라스는 이 함수들을 자동으로 텐서플로 함수로 변환해줌.  
> 파이썬 값으로 텐서플로 함수를 여러 번 호출하면 프로그램이 느려지고 메모리가 많이 잡아먹힘.  
**층의 뉴런 개수와 같은 하이퍼파라미터처럼 몇 개의 고유값이 있는 경우에만 매개변수 값에 파이썬 값을 사용해야 함.**  
그래야 텐서플로가 모델을 잘 최적화할 수 있음.

---
## 텐서플로 함수 사용 방법
>1. 넘파이나 표준 라이브러리를 포함해서 다른 라이브러리를 호출하면 트레이싱 과정에서 실행됨. 트레이싱 과정에서 코드가 실행되는 것을 원하지 않는다면 **np.sum() 대신에 tf.reduce_sum()을, sorted() 내장 함수 대신에 tf.sort()** 와 같이 사용하면 됨.

>2. 다른 파이썬 함수나 텐서플로 함수를 호출할 수 있음. 하지만 텐서플로가 계산 그래프에 있는 이 함수들의 연산을 감지하므로 동일한 규칙을 따름. 이런 함수들은 @tf.function 데코레이터를 적용할 필요가 없음 (잘 이해가 안감..)

>3. 함수에서 텐서플로 변수를 만든다면 처음 호출될 때만 수행되어야 함. 아니면 예외가 발생함. **일반적으로 텐서플로 함수밖에서 변수를 생성하는 것이 좋음.** (예를 들어 사용자 정의 층의 build()메서드). **변수에 새로운 값을 할당하려면 = 연산자 대신에 assign() 메서드를 이용**

>4. 파이썬 함수의 소스 코드는 텐서플로에서 사용 가능해야 함. 만약 소스 코드를 사용할 수 없다면 그래프 생성 과정이 실패하거나 일부 기능을 사용할 수 없음.

>5. 텐서플로는 텐서나 데이터셋을 순회하는 for문만 감지함. 따라서 **for i in range(x) 대신에 for i in tf.range(x)** 를 사용해야 함. 그렇지 않으면 이 반복문이 그래프에 표현되지 못함. 대신 트레이싱 단계에서 실행됨.

>6. 성능면에서 반복문보다는 가능한 한 벡터화된 구현을 사용하는 것이 좋음